In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df = pd.read_csv('/content/drive/MyDrive/ml_project/illegal_fishing_train.csv')
#change path accordingly

In [4]:
df.describe()

,mmsi,timestamp,distance_from_shore,distance_from_port,speed,course,lat,lon,labels
count,8.388600e+05,8.388600e+05,8.388600e+05,8.388600e+05,838857.000000,838857.000000,838860.000000,838860.000000,838860.000000
mean,6.473890e+13,1.424273e+09,2.978600e+04,4.754523e+04,2.297752,183.344641,52.284092,5.946398,-0.950138
std,4.463738e+13,3.866942e+07,6.801536e+04,8.329854e+04,3.747304,114.404061,7.487147,11.325362,0.247370
min,1.252340e+12,1.325376e+09,0.000000e+00,0.000000e+00,0.000000,0.000000,-83.484398,-167.559769,-1.000000
25%,1.985950e+13,1.394920e+09,0.000000e+00,9.999755e+02,0.000000,75.699997,52.099167,-2.001655,-1.000000
50%,6.960990e+13,1.432542e+09,0.000000e+00,1.004963e+04,0.000000,200.000000,53.779861,4.585558,-1.000000
75%,1.124530e+14,1.456679e+09,4.002401e+04,6.985528e+04,3.100000,283.399994,57.393181,18.418943,-1.000000
max,1.414320e+14,1.480032e+09,2.252376e+06,1.181676e+07,102.300003,405.100006,77.244652,177.426651,1.000000


In [7]:
#dropping rows with null values
df=df.dropna()


In [9]:
df.describe

<bound method NDFrame.describe of                 mmsi   timestamp  distance_from_shore  distance_from_port  \
0       1.214260e+14  1392604788          62239.93750        69998.281250   
1       1.252340e+12  1456971800          42952.41016        52477.277340   
2       1.217390e+14  1428161768              0.00000         2999.926270   
3       1.414320e+14  1438085847              0.00000          999.975464   
4       1.985950e+13  1438331947          11401.47461        49647.546880   
...              ...         ...                  ...                 ...   
838855  5.110130e+13  1372087434              0.00000        10049.628910   
838856  5.145480e+12  1440923683              0.00000         1999.950928   
838857  3.491170e+13  1446406724         136689.73440       181907.593800   
838858  1.985950e+13  1453508316              0.00000          999.975464   
838859  7.726190e+13  1376651306          50208.32813        66526.554690   

        speed      course        lat     

In [ ]:
#installing necessary libraries to run thundersvm

In [10]:
!wget https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb
!ls  # Check if required cuda 9.0 amd64-deb file is downloaded
!dpkg -i cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb
!ls /var/cuda-repo-9-0-local | grep .pub
!apt-key add /var/cuda-repo-9-0-local/7fa2af80.pub
!apt-get update
!sudo apt-get install cuda-9.0

--2023-11-19 09:06:15--  https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb
Resolving developer.nvidia.com (developer.nvidia.com)... 152.195.19.142
Connecting to developer.nvidia.com (developer.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.nvidia.com/downloads/compute/cuda/9.0/prod/local_installers/cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb [following]
--2023-11-19 09:06:15--  https://developer.nvidia.com/downloads/compute/cuda/9.0/prod/local_installers/cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb
Reusing existing connection to developer.nvidia.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.0/secure/Prod/local_installers/cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64.deb?S9dXTNmrF5EyBAePnlpNSoWyvyPq9UYvw2xXNw93alKB4rWHk_umnBnMYqivcijSV9_BjiudZwGP5-slH3t9C

In [11]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2017 NVIDIA Corporation
Built on Fri_Sep__1_21:08:03_CDT_2017
Cuda compilation tools, release 9.0, V9.0.176


In [12]:
!pip install thundersvm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.4/507.4 kB 8.6 MB/s eta 0:00:00


In [18]:
#replacing negative valued label to prevent error
df['labels'] = df['labels'].replace(-1, 2)

In [19]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from thundersvm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

#running with al features
target_column = 'labels'
selected_columns = ['mmsi', 'timestamp', 'distance_from_shore', 'distance_from_port',
       'speed', 'course', 'lat', 'lon']


X = df[selected_columns]
y = df[target_column]

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
#from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

#train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7, stratify=y)

#defining classifier
svm_classifier = SVC(class_weight='balanced')

# parameter grid
param_grid = {
    'svm__C': [0.1, 1, 10],
    'svm__kernel': ['linear', 'rbf'],
    'svm__gamma': ['scale', 'auto'],
}

# pipeline
pipeline = Pipeline([
    ('scaler', MinMaxScaler()),  # Use MinMaxScaler for feature scaling
    ('svm', svm_classifier),
])

kf = StratifiedKFold(n_splits=3, random_state = 7, shuffle=True)

# defining the gridsearch
grid_search = GridSearchCV(pipeline, param_grid, cv=kf, scoring='f1_macro')

# running gridsearch
grid_search.fit(X_train, y_train)

# Get the best parameters and best estimator from the grid search
best_params = grid_search.best_params_
best_estimator = grid_search.best_estimator_

# Make predictions on the test set
y_pred = best_estimator.predict(X_test)

# Print classification report and confusion matrix
print("Best Parameters:", best_params)
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
18 fits failed out of a total of 36.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
18 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/pipeline.py", line 405, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/usr/local/lib/python3.10/dist-packages/thundersvm/thundersvm.py", line 117, in fit
    fit(X, y, solver_type, kernel)
  File "/usr/local/lib/python3.10/dist-pa

Best Parameters: {'svm__C': 10, 'svm__gamma': 'auto', 'svm__kernel': 'rbf'}

Classification Report:
               precision    recall  f1-score   support

           0       0.13      0.86      0.23      6047
           1       0.05      0.90      0.09      1159
           2       1.00      0.65      0.79    160566

    accuracy                           0.66    167772
   macro avg       0.39      0.81      0.37    167772
weighted avg       0.96      0.66      0.77    167772


Confusion Matrix:
 [[  5229    762     56]
 [    88   1041     30]
 [ 34478  21025 105063]]


In [ ]:
#running with selected features

target_column = 'labels'
selected_columns = ['timestamp',  'lon']

X = df[selected_columns]
y = df[target_column]

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7, stratify=y)

svm_classifier = SVC(class_weight='balanced')
param_grid = {
    'svm__C': [0.1, 1, 10],
    'svm__kernel': ['linear', 'rbf'],
    'svm__gamma': ['scale', 'auto'],
}

#pipeline
pipeline = Pipeline([
    ('scaler', MinMaxScaler()),
    ('svm', svm_classifier),
])

kf = StratifiedKFold(n_splits=3, random_state = 7, shuffle=True)

grid_search = GridSearchCV(pipeline, param_grid, cv=kf, scoring='accuracy')

grid_search.fit(X_train, y_train)


best_params = grid_search.best_params_
best_estimator = grid_search.best_estimator_


y_pred = best_estimator.predict(X_test)


print("Best Parameters:", best_params)
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", cbar=False,
            xticklabels=best_estimator.classes_, yticklabels=best_estimator.classes_)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()
